# Assessment 1: BugFinder
### Intro
This notebook is to be the record of completion for Assessment 1: Machine Learning.
### Scenario
Develop a model to be used with a hand-held hyperspectral camera system to identify harmful pests on containers and vessels entering the country, with the aim of preventing those pests from establishing themselves in this country and destroying native wildlife. This project will use a standard camera to develop a proof of concept for this system.

## Dataset Preprocessing & Organisation
#### Locating Dataset
I used Kaggle to research datasets containing images of a variety of different insects, and located two potentially suitable, pre-labelled data sets:
- https://www.kaggle.com/datasets/shameinew/insect-images-with-scientific-names
- https://www.kaggle.com/datasets/rtlmhjbn/ip02-dataset
#### Preprocessing Dataset
The following code 

1. Research and locate a suitable dataset that contains images of hitchhiker pests and their associated environments.
2. Preprocess the dataset to ensure consistency in image dimensions and data quality.
3. Organize the dataset into appropriate directories for training, validation, and testing.

## Creating an ML Model:

1. Utilize the fastai library to create an image classification model.
2. Choose an appropriate deep learning architecture (e.g., CNN) for the model.
3. Train the model using the training dataset, considering hyperparameter tuning.
4. Monitor training progress and adjust if necessary.

## Model Scoring:

1. Use the trained model to predict pest species in a given set of images from the validation dataset.
2. Evaluate the model's performance using appropriate metrics (e.g., accuracy, precision, recall).
3. Visualize the model's predictions and actual labels.

## Validation and Test Datasets:

1. Create a validation dataset that was not used during training to assess the model's generalization ability.
2. Ensure that the validation dataset contains images with varying conditions and perspectives.
3. Additionally, prepare a separate test dataset for final model evaluation.

## ML Evaluations:

1. Apply the trained model to the test dataset to evaluate its performance on previously unseen data.
2. Analyze the model's predictions, misclassifications, and potential areas of improvement.
3. Summarize the assessment of the model's capabilities and limitations.